# 서울시 구별 주유소 가격 확인
### 하는데 위도, 경도를 Googlemap이 아니라 Kakao에서 가져오기

In [1]:
import requests
from urllib.parse import quote
import numpy as np
import pandas as pd
import folium
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [2]:
key_fd = open("./keys/kakaomapkey.txt", mode = "r", encoding = "utf8")
kmap_key = key_fd.read(100)
key_fd.close()

In [3]:
stations = pd.read_csv('04. stations.csv' , sep="," , encoding='utf8')
del stations['Unnamed: 0']

top10

In [4]:
stations.sort_values(by='가격',ascending=False, inplace=True)
top10 = stations.head(10)
top10

,이름,주소,가격,셀프,상표,구
303,서남주유소,서울 중구 통일로 30,2231.0,N,SK에너지,중구
302,에스씨(주) 역전주유소,서울 중구 퇴계로 15,2138.0,N,GS칼텍스,중구
301,필동주유소,서울 중구 퇴계로 196 (필동2가),2109.0,N,GS칼텍스,중구
300,SK에너지(주) 퇴계로주유소,서울 중구 퇴계로 228 (필동2가),2099.0,N,SK에너지,중구
266,서계주유소,서울 용산구 청파로 367 (서계동),2089.0,N,GS칼텍스,용산구
492,뉴서울(강남),서울 강남구 언주로 716 (논현동),2025.0,N,SK에너지,강남구
291,(주)대양씨앤씨 사직주유소,서울 종로구 사직로 65 (사직동),1998.0,N,GS칼텍스,종로구
265,한남지점,서울 용산구 한남대로21길 4 (한남동),1998.0,N,SK에너지,용산구
290,(주)중앙에너비스 혜화주유소,서울 종로구 창경궁로35길 1,1998.0,N,SK에너지,종로구
264,에너비스,서울 용산구 한남대로 82 (한남동),1998.0,N,SK에너지,용산구


bot10

In [5]:
stations.sort_values(by='가격',ascending=True, inplace=True)
bot10 = stations.head(10)
bot10

,이름,주소,가격,셀프,상표,구
75,만남의광장주유소,서울 서초구 양재대로12길 73-71,1297.0,Y,알뜰(ex),서초구
320,강서오곡셀프주유소,서울 강서구 벌말로 254 (오곡동),1297.0,Y,SK에너지,강서구
321,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),1308.0,N,알뜰주유소,강서구
322,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1308.0,Y,알뜰주유소,강서구
323,방화주유소,서울 강서구 방화동로 58-2 (방화동),1313.0,N,SK에너지,강서구
324,스카이주유소,서울 강서구 개화동로 457 (방화동),1315.0,Y,S-OIL,강서구
268,타이거주유소,서울 은평구 수색로 188 (증산동),1315.0,Y,SK에너지,은평구
267,(주)명연에너지 수색훼미리주유소,서울 은평구 수색로 236 (수색동),1315.0,Y,현대오일뱅크,은평구
127,원천주유소,서울 성북구 돌곶이로 142 (장위동),1315.0,N,알뜰주유소,성북구
183,현대주유소,서울 양천구 남부순환로 372 (신월동),1316.0,Y,S-OIL,양천구


카카오에서 위도, 경도 가져오기

In [6]:
addr = '서울특별시 강서구 강서로 154 (화곡동)'
url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+quote(addr)

In [7]:
result = requests.get(url, headers={'Authorization':'KakaoAK '+kmap_key}).json()
result

{'documents': [{'address': {'address_name': '서울 강서구 화곡동 1076-14',
    'b_code': '1150010300',
    'h_code': '1150059000',
    'main_address_no': '1076',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '강서구',
    'region_3depth_h_name': '화곡본동',
    'region_3depth_name': '화곡동',
    'sub_address_no': '14',
    'x': '126.8414715329',
    'y': '37.5407680113225'},
   'address_name': '서울 강서구 강서로 154',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 강서구 강서로 154',
    'building_name': '힐탑빌딩',
    'main_building_no': '154',
    'region_1depth_name': '서울',
    'region_2depth_name': '강서구',
    'region_3depth_name': '화곡동',
    'road_name': '강서로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.8414715329',
    'y': '37.5407680113225',
    'zone_no': '07727'},
   'x': '126.8414715329',
   'y': '37.5407680113225'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [8]:
lat = result['documents'][0]['y']
lng = result['documents'][0]['x']

top10, bot10에 lat, lng 추가하기

In [9]:
top10_lat = [] ; top10_lng = []

for i in top10.index:
    addr = top10['주소'][i]
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+quote(addr)
    result = requests.get(url, headers={'Authorization':'KakaoAK '+kmap_key}).json()

    top10_lat.append(float(result['documents'][0]['y']))
    top10_lng.append(float(result['documents'][0]['x']))

bot10_lat = [] ; bot10_lng = []

for i in bot10.index:
    addr = bot10['주소'][i]
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+quote(addr)
    result = requests.get(url, headers={'Authorization':'KakaoAK '+kmap_key}).json()

    bot10_lat.append(float(result['documents'][0]['y']))
    bot10_lng.append(float(result['documents'][0]['x']))

top10['lat'] = top10_lat
top10['lng'] = top10_lng
bot10['lat'] = bot10_lat
bot10['lng'] = bot10_lng

Folium으로 지도 그리기

In [16]:
mapping = folium.Map( [top10.lat.mean(), top10.lng.mean()], zoom_start = 10.5)
# 아이콘 https://getbootstrap.com/docs/3.4/components/
for i in top10.index:
    folium.Marker( [top10.lat[i], top10.lng[i]], 
    popup = top10['주소'][i], tooltip = top10['이름'][i] +'\n셀프: '+top10['셀프'][i]','+top10['가격'],
    icon=folium.Icon(color = 'red', icon='remove')).add_to(mapping)

    folium.CircleMarker( [top10.lat[i], top10.lng[i]] , radius = 15, color = 'red', fill_color = 'purple').add_to(mapping)

for i in bot10.index:
    folium.Marker( [bot10.lat[i], bot10.lng[i]], 
    popup = bot10['주소'][i], tooltip =  bot10['이름'][i] +'\n셀프: '+bot10['셀프'][i]+','+bot10['가격'],
    icon=folium.Icon(color = 'green', icon='heart')).add_to(mapping)

    folium.CircleMarker( [bot10.lat[i], bot10.lng[i]] , radius = 15, color = 'green', fill_color = 'blue').add_to(mapping)
mapping


In [17]:
# 한글이 나오기도 하고 안 나오기도 함
mapping.save('top10 bot10 한글.html')